# Average Relative Brier Score - Ensemble of 50 Model Chains

This notebook compares the Average Relative Brier scores of an ensemble of 50 Model Chains where we propagate the true target values.

In [1]:
import numpy as np
import pandas as pd
import itertools
import random

from sklearn.ensemble import RandomForestRegressor, RandomForestClassifier
from sklearn.metrics import mean_squared_error as mse, brier_score_loss
from chaining import Chain
import os
from scipy.stats import mode

In [2]:
def generate_permutations_with_order(variables, pairs_or_groups, num_permutations, constrained_elements=None, shuffle_at_end=None, random_state=None):
    random.seed(random_state)
    permutations_list = [list(variables)]  # Add the original order only once as a list of strings
    
    while len(permutations_list) < num_permutations + 1: # +1 because the original order is counted too
        perm = list(random.sample(variables, len(variables)))
        valid = True
        
        for pair_or_group in pairs_or_groups:
            idxs = [perm.index(var) for var in pair_or_group]
            if sorted(idxs) != idxs:
                valid = False
                break
        
        if valid:
            if constrained_elements:
                # Check if all constrained elements are present in the first positions of the permutation
                if all(elem in perm[:len(constrained_elements)] for elem in constrained_elements):
                    permutations_list.append(perm)
    
    # Shuffle the positions of variables specified to be shuffled at the end
    if shuffle_at_end:
        for idx, perm in enumerate(permutations_list[1:], start=1):  # Start from index 1 because original order shouldn't be shuffled
            for variable in shuffle_at_end:
                if variable in perm:
                    perm.remove(variable)
                    perm.insert(random.randint(0, len(perm)), variable)
    
    return permutations_list

In [3]:
def missingness_stratified_cv(df, N_FOLDS=5, random_state=None):
    # Add seed for reproducibility of the predictions (to get the same scores each time we run the code)
    np.random.seed(random_state)

    # Initial complete-case test fold assignment
    cv = pd.Series(np.nan, index=df.index)
    i_cc = (df.isna().sum(axis=1) == 0) # Complete cases
    cv.iloc[i_cc] = np.random.randint(low=0, high=N_FOLDS, size=i_cc.sum())

    # Go over columns from most missing to least missing
    for j in df.isna().sum().argsort()[::-1]:
        # Instances i that are not assigned yet but for which df[i,j] is observed
        i_tbf = (cv.isna()) & (~df.iloc[:,j].isna()) # to be filled
        # Fill them randomly
        cv.iloc[i_tbf] = np.random.randint(low=0, high=N_FOLDS, size=i_tbf.sum())

    return cv

In [4]:
def missingness_and_categorical_stratified_cv(df, N_FOLDS=5, random_state=None):
    # Add seed for reproducibility of the predictions (to get the same scores each time we run the code)
    np.random.seed(random_state)

    # Initial complete-case test fold assignment
    cv = pd.Series(np.nan, index=df.index)
    i_cc = (df.isna().sum(axis=1) == 0) # Complete cases
    cv.iloc[i_cc] = np.random.randint(low=0, high=N_FOLDS, size=i_cc.sum())

    # Stratify categorical variables
    for col in df.select_dtypes(include=['category']):
        counts = df[col].value_counts(normalize=True)
        for category in counts.index:
            idx = df[col] == category
            cv[idx] = cv[idx].fillna(np.random.choice(np.where(idx)[0], size=int(counts[category] * N_FOLDS), replace=False))

    # Go over columns from most missing to least missing
    for j in df.isna().sum().argsort()[::-1]:
        # Instances i that are not assigned yet but for which df[i,j] is observed
        i_tbf = (cv.isna()) & (~df.iloc[:,j].isna()) # to be filled
        # Fill them randomly
        cv.iloc[i_tbf] = np.random.randint(low=0, high=N_FOLDS, size=i_tbf.sum())

    return cv

In [5]:
def normalized_mean_squared_error(true, pred, train):
    num = mse(true, pred)
    mean_value = np.mean(train)
    mean = np.full_like(true, mean_value)
    den = mse(true, mean)
    nmse_loss = num/den
    
    return nmse_loss

In [6]:
# Function to reorder columns of dataframes
def reorder_columns(dataframes):
    column_order = dataframes[0].columns
    reordered_dataframes = [df[column_order] for df in dataframes]
    return reordered_dataframes

In [7]:
# Function to average dataframes
def average_dataframes(dataframes):
    concatenated_df = pd.concat(dataframes)
    # Group by index and calculate the mode for object columns and mean for other types
    averaged_df = concatenated_df.groupby(concatenated_df.index).agg(lambda x: x.mode()[0] if x.dtype == 'O' else x.mean())
    return averaged_df

-----

In [8]:
possible_paths = [
    'C:/Users/lenne/OneDrive/Documenten/Master of Statistics and Data Science/2023-2024/Master thesis/Thesis_Sofia_Lennert/new_data',
    'C:/Users/anaso/Desktop/SOFIA MENDES/KU Leuven/Master Thesis/Thesis_Sofia_Lennert/new_data'
]

# File name
file = 'merged_data.csv'

# Find full paths to the CSV files
path = next((f'{path}/{file}' for path in possible_paths if os.path.exists(f'{path}/{file}')), None)

# Resulting DataFrame will have aggregated data from all four datasets based on the specific_column
pd.set_option('display.max_columns', None)

data = pd.read_csv(path)

def bin_column(value):
    if value in [0, 1, 2, 3]:
        return str(value)
    else:
        return '4+'
data['NRELAP'] = data['NRELAP'].apply(bin_column)

#data

In [9]:
# Choice of target variables, and listed already in the chain order 
variables = ['KFSS_M-2y', 'KFSS_P-2y', 'EDSS-2y', 'T25FW-2y', 'NHPT-2y', 'P_R36-SF12-after', 'M_R36-SF12-after', 
             'SES_after', 'SLEC_after', 'KFSS_M-after_2y', 'KFSS_P-after_2y', 'EDSS-after_2y', 'NRELAP', 'CESEV']

# Choice of input variables
columns_to_keep = ['AGE', 'SEX', 'RACE', 'CONTINENT', 'MHDIAGN', 'CARDIO', 'URINARY', 'MUSCKELET', 'FATIGUE', 
                    'NHPT-before', 'PASAT_2s-before', 'PASAT_3s-before', 'SDMT-before', 'T25FW-before', 'SLEC_before','SES_before',
                    'BDI-before', 'EDSS-before', 'KFSS_M-before', 'KFSS_P-before', 'M_R36-SF12-before',
                	'P_R36-SF12-before', 'R36-SF12-before_Ind', 'T-before','P-before','N-before']

features = data[columns_to_keep]
#features

targets = data[variables]

In [10]:
# Use one-hot encoding for categorical and binary input variables
object_columns = features.select_dtypes(include=['object'])
features = pd.get_dummies(features, columns=object_columns.columns, dtype=int)
#features.head()

In [11]:
# Generate 50 permutations of the target variables: the chain orders for the different chains in the ensemble
pairs_or_groups = [['KFSS_M-2y', 'EDSS-2y'], ['KFSS_P-2y', 'EDSS-2y'], ['KFSS_M-after_2y', 'EDSS-after_2y'], ['KFSS_P-after_2y', 'EDSS-after_2y']]  # Specify the pairs or groups
order_constraint = ['KFSS_M-2y', 'KFSS_P-2y', 'EDSS-2y', 'T25FW-2y', 'NHPT-2y']
shuffle_at_end = ['NRELAP', 'CESEV']  # Specify variables to be shuffled at the end
num_permutations = 49  # Specify how many random permutations you want
random_state = 42

random_permutations = generate_permutations_with_order(variables, pairs_or_groups, num_permutations, order_constraint, shuffle_at_end, random_state)

# Print the original order followed by all the random permutations
for idx, perm in enumerate(random_permutations, start=0):
    print(f"Permutation {idx}: {', '.join(perm)}")

Permutation 0: KFSS_M-2y, KFSS_P-2y, EDSS-2y, T25FW-2y, NHPT-2y, P_R36-SF12-after, M_R36-SF12-after, SES_after, SLEC_after, KFSS_M-after_2y, KFSS_P-after_2y, EDSS-after_2y, NRELAP, CESEV
Permutation 1: T25FW-2y, KFSS_M-2y, NHPT-2y, KFSS_P-2y, EDSS-2y, NRELAP, KFSS_M-after_2y, SLEC_after, M_R36-SF12-after, CESEV, P_R36-SF12-after, KFSS_P-after_2y, EDSS-after_2y, SES_after
Permutation 2: NHPT-2y, NRELAP, KFSS_P-2y, KFSS_M-2y, T25FW-2y, CESEV, EDSS-2y, M_R36-SF12-after, SES_after, SLEC_after, KFSS_P-after_2y, P_R36-SF12-after, KFSS_M-after_2y, EDSS-after_2y
Permutation 3: NHPT-2y, KFSS_P-2y, KFSS_M-2y, T25FW-2y, EDSS-2y, KFSS_P-after_2y, M_R36-SF12-after, CESEV, P_R36-SF12-after, SES_after, SLEC_after, KFSS_M-after_2y, EDSS-after_2y, NRELAP
Permutation 4: KFSS_P-2y, KFSS_M-2y, NHPT-2y, T25FW-2y, EDSS-2y, NRELAP, KFSS_M-after_2y, P_R36-SF12-after, CESEV, KFSS_P-after_2y, SES_after, SLEC_after, EDSS-after_2y, M_R36-SF12-after
Permutation 5: T25FW-2y, KFSS_M-2y, KFSS_P-2y, EDSS-2y, NHPT-2y, 

In [12]:
ordered_targets = random_permutations[0]

In [13]:
# Set random state for reproducibility
random_state = 42
N_FOLDS = 5

In [14]:
# Generate CV folds
cv=missingness_and_categorical_stratified_cv(targets, N_FOLDS, random_state)
cv = cv.to_frame(name="CV Fold")

---

## Chain with *true* values propagated

In [14]:
y_pred_chains = []
y_pred_prob_list_chain = []
y_test_list = [[] for _ in range(N_FOLDS)]  # Initialize y_test_list with empty lists for each fold index
y_train_list = [[] for _ in range(N_FOLDS)] 
yi_test_dummies_list = [[] for _ in range(N_FOLDS)]
yi_train_dummies_list = [[] for _ in range(N_FOLDS)]


# Iterate over each chain ordering
for ordered_targets_chain in random_permutations:
    y_pred_list = []  # List to store predictions for this chain
    y_pred_prob_list = []
    
    features_cv = pd.merge(features, pd.DataFrame(cv), left_index=True, right_index=True)
    targets_cv = pd.merge(data[ordered_targets_chain], pd.DataFrame(cv), left_index=True, right_index=True)

    # Fit and predict for each fold for this chain
    for i in range(0, N_FOLDS): 
        Xi_train = features_cv[features_cv['CV Fold'] != i].drop(["CV Fold"], axis=1)
        Xi_test = features_cv[features_cv['CV Fold'] == i].drop(["CV Fold"], axis=1)
        yi_train = targets_cv[targets_cv['CV Fold'] != i].drop(["CV Fold"], axis=1)
        yi_test = targets_cv[targets_cv['CV Fold'] == i].drop(["CV Fold"], axis=1)

        # One hot encode categorical targets of test set to be able to compute brier score
        subset_yi_test = yi_test.select_dtypes(include=['object'])
        yi_test_dummies = pd.get_dummies(subset_yi_test, columns=subset_yi_test.columns, dtype=int)
        subset_yi_train = yi_train.select_dtypes(include=['object'])
        yi_train_dummies = pd.get_dummies(subset_yi_train, columns=subset_yi_train.columns, dtype=int)


        chain = Chain(
            model_reg=RandomForestRegressor(random_state=random_state),
            model_clf=RandomForestClassifier(random_state=random_state),
            propagate="true",
        )


        chain.fit(Xi_train, yi_train, target_types=None)
        y_pred = chain.predict(Xi_test)
        y_pred_prob = chain.predict_proba(Xi_test)
        y_pred_list.append(pd.DataFrame(y_pred, columns=yi_test.columns, index=yi_test.index))
        y_pred_prob_list.append(y_pred_prob)
    
        # Append yi_test to the corresponding fold index in y_test_list
        y_test_list[i].append(yi_test)  
        y_train_list[i].append(yi_train)
        yi_test_dummies_list[i].append(yi_test_dummies)
        yi_train_dummies_list[i].append(yi_train_dummies)

    y_pred_chains.append(y_pred_list)
    y_pred_prob_list_chain.append(y_pred_prob_list)
    print("Permutation done")

Permutation done
Permutation done
Permutation done
Permutation done
Permutation done
Permutation done
Permutation done
Permutation done
Permutation done
Permutation done
Permutation done
Permutation done
Permutation done
Permutation done
Permutation done
Permutation done
Permutation done
Permutation done
Permutation done
Permutation done
Permutation done
Permutation done
Permutation done
Permutation done
Permutation done
Permutation done
Permutation done
Permutation done
Permutation done
Permutation done
Permutation done
Permutation done
Permutation done
Permutation done
Permutation done
Permutation done
Permutation done
Permutation done
Permutation done
Permutation done
Permutation done
Permutation done
Permutation done
Permutation done
Permutation done
Permutation done
Permutation done
Permutation done
Permutation done
Permutation done


In [15]:
concatenated_dfs_chain = []

for y_pred_prob_list in y_pred_prob_list_chain:
    concatenated_dfs_fold = []
    # Iterate over each pair of arrays
    for j, fold in enumerate(y_pred_prob_list):
        dfs = []
        len_array = 0
        
        for i, array in enumerate(fold):
            # Convert array to DataFrame
            col = yi_test_dummies_list[j][0].columns[len_array:len_array+len(array[0])]
            df = pd.DataFrame(array, columns=col, index=yi_test_dummies_list[j][0].index)
            dfs.append(df)
            len_array += len(array[0])
        
        # Concatenate DataFrames
        concatenated_df = pd.concat(dfs, axis=1)
        concatenated_dfs_fold.append(concatenated_df)

    concatenated_dfs_chain.append(concatenated_dfs_fold)

In [16]:
transposed_list_cat = list(zip(*concatenated_dfs_chain))

In [17]:
# Each element of transposed_list is a tuple containing dataframes from the same position in each inner list
reorganized_list_cat = [list(df_tuple) for df_tuple in transposed_list_cat]

In [18]:
reordered_reorganized_list_cat = [reorder_columns(dataframes) for dataframes in reorganized_list_cat]

In [19]:
averaged_dataframes_list_cat = [average_dataframes(dataframes) for dataframes in reordered_reorganized_list_cat]

In [20]:
transposed_list_all = list(zip(*y_pred_chains))

reorganized_list_all = [list(df_tuple) for df_tuple in transposed_list_all]

reordered_reorganized_list_all = [reorder_columns(dataframes) for dataframes in reorganized_list_all]

averaged_dataframes_list_all = [average_dataframes(dataframes) for dataframes in reordered_reorganized_list_all]

In [21]:
transposed_test_list = list(zip(*y_test_list))

reorganized_test_list = [list(df_tuple) for df_tuple in transposed_test_list]

In [22]:
transposed_train_list = list(zip(*y_train_list))

reorganized_train_list = [list(df_tuple) for df_tuple in transposed_train_list]

In [23]:
# Remove rows in y_test and y_pred where the variable in question is missing in y_test (since without it, it is not possible to calculate the score)
y_pred_list = averaged_dataframes_list_all.copy()
y_test_list = reorganized_test_list[0]

y_test_cv = []
y_pred_cv = []

for j in range(len(y_test_list)):  # 5
    y_test_targ = []
    y_pred_targ = []
    nvar=y_test_list[0].shape[1]

    for i in range(0, nvar):  # or (1, 5)
        missing_rows_mask = y_test_list[j].iloc[:, i].isna()
        y_test = y_test_list[j].iloc[:, i][~missing_rows_mask]
        y_pred = y_pred_list[j].iloc[:, i][~missing_rows_mask]
        
        y_test_targ.append(y_test)
        y_pred_targ.append(y_pred)
    
    y_test_cv.append(y_test_targ)
    y_pred_cv.append(y_pred_targ)

In [24]:
reorganized_train_list_first=reorganized_train_list[0]

In [25]:
scores_with_std = []

# Iterate over each outcome variable in the folds
for variable_name in variables: 
    variable_scores = []
    
    # Check if the target variable is numerical or categorical
    if y_test_cv[0][variables.index(variable_name)].dtype.kind in 'bifc':
        # Compute scores for the variable across all folds
        for fold_index in range(len(y_test_cv)):
            y_test = y_test_cv[fold_index][variables.index(variable_name)] 
            y_pred = y_pred_cv[fold_index][variables.index(variable_name)] 
            y_train = reorganized_train_list_first[fold_index][variable_name]

            score = normalized_mean_squared_error(y_test, y_pred, y_train)
            variable_scores.append(score)
        
        # Compute the average score for the variable across all folds
        variable_avg_score = np.mean(variable_scores)
        
        # Compute the standard deviation for the variable across all folds
        variable_std_score = np.std(variable_scores)
        
        # Append the tuple with three elements to the scores_with_std list
        scores_with_std.append((variable_name, variable_avg_score, variable_std_score))

num_normalized_brier=[]
num_std_brier=[]
# Print the scores with average and standard deviation along with variable names
print("Scores for each outcome (chain - true values):")
for variable_name, avg_score, std_score in scores_with_std:
    print(f"{variable_name}: {avg_score:.2f} (± {std_score:.2f})")
    num_normalized_brier.append(avg_score)
    num_std_brier.append(std_score)

Scores for each outcome (chain - true values):
KFSS_M-2y: 0.19 (± 0.02)
KFSS_P-2y: 0.25 (± 0.03)
EDSS-2y: 0.12 (± 0.01)
T25FW-2y: 0.27 (± 0.09)
NHPT-2y: 0.46 (± 0.11)
P_R36-SF12-after: 0.30 (± 0.01)
M_R36-SF12-after: 0.43 (± 0.03)
SES_after: 0.31 (± 0.05)
SLEC_after: 0.35 (± 0.04)
KFSS_M-after_2y: 0.33 (± 0.05)
KFSS_P-after_2y: 0.48 (± 0.05)
EDSS-after_2y: 0.23 (± 0.02)


In [26]:
yi_train_dummies_avg = []
i=0

for yi_train_dummies_fold in yi_train_dummies_list:
    # Calculate the percentage of 1s in each column
    yi_test_dummies_avg_fold=[]

    for yi_train_dummies_chain in yi_train_dummies_fold:

        percentages = yi_train_dummies_chain.sum() / len(yi_train_dummies_chain)

        yi_train_dummies_avg_chain = pd.DataFrame(0, index=yi_test_dummies_list[i][0].index, columns=yi_train_dummies_chain.columns)

        # Replace values in each column with the corresponding percentage
        for col in yi_train_dummies_avg_chain.columns:
            yi_train_dummies_avg_chain[col] = yi_train_dummies_avg_chain[col].apply(lambda x: percentages[col])
        
        yi_test_dummies_avg_fold.append(yi_train_dummies_avg_chain)

    i += 1    
    yi_train_dummies_avg.append(yi_test_dummies_avg_fold)

In [27]:
transposed_dummy_avg_list = list(zip(*yi_train_dummies_avg))

reorganized_dummy_avg_list = [list(df_tuple) for df_tuple in transposed_dummy_avg_list]

In [28]:
transposed_dummy_list = list(zip(*yi_test_dummies_list))

reorganized_dummy_list = [list(df_tuple) for df_tuple in transposed_dummy_list]

In [29]:
reorganized_dummy_avg_list_first=reorganized_dummy_avg_list[0]
reorganized_dummy_list_first=reorganized_dummy_list[0]

In [30]:
scores_with_std = []
variables_cat = reorganized_dummy_list_first[0].columns
avg_brier_score = []
avg_baseline_score = []
cat_normalized_brier=[]

# Create a dictionary to store the scores for variables with the same letters before the '_'
brier_scores_dict = {}
baseline_scores_dict = {}

# Iterate over each outcome variable in the folds
for level_name in variables_cat: 
    variable_scores = []
    brier_scores = []
    baseline_scores = []
    
    # Compute scores for the variable across all folds
    for fold_index in range(len(yi_test_dummies_list)):
        y_test = reorganized_dummy_list_first[fold_index][level_name] 
        y_prob = averaged_dataframes_list_cat[fold_index][level_name] 
        y_prob_avg = reorganized_dummy_avg_list_first[fold_index][level_name] 
        
        # Compute the Brier score
        brier_score = brier_score_loss(y_test, y_prob)
        N_brier_score = brier_score#*N
        brier_baseline = brier_score_loss(y_test, y_prob_avg)
        N_brier_baseline = brier_baseline#*N

        # Append the Brier score to the variable scores list
        brier_scores.append(N_brier_score)
        baseline_scores.append(N_brier_baseline)
    
    # Check if the variable name has letters before the '_'
    prefix = level_name.split('_')[0]
    
    # Add the Brier scores to the dictionary based on the prefix
    if prefix in brier_scores_dict:
        brier_scores_dict[prefix].extend(brier_scores)
    else:
        brier_scores_dict[prefix] = brier_scores

    if prefix in baseline_scores_dict:
        baseline_scores_dict[prefix].extend(baseline_scores)
    else:
        baseline_scores_dict[prefix] = baseline_scores


# Compute the average of Brier score for each prefix
for prefix, scores in brier_scores_dict.items():
    sum_score = np.sum(scores)
    avg_brier_score.append((prefix, sum_score))

for prefix, scores in baseline_scores_dict.items():
    sum_score = np.sum(scores)
    avg_baseline_score.append((prefix, sum_score))


normalized_score_list = []
for i in range(len(avg_brier_score)):
    normalized_score = avg_brier_score[i][1]/avg_baseline_score[i][1]
    cell = (avg_brier_score[i][0], normalized_score)
    normalized_score_list.append(cell)


print("Normalized Brier scores for each categorical variable:")
for prefix, avg_score in normalized_score_list:
    print(f"{prefix}: {avg_score:.2f} ")
    cat_normalized_brier.append(avg_score)

Normalized Brier scores for each categorical variable:
NRELAP: 1.43 
CESEV: 1.02 


In [31]:
# Concatenate normalized brier scores for all variables (both numerical and categorical) 
combined_normalized_brier = np.concatenate((num_normalized_brier, cat_normalized_brier))
combined_normalized_brier

array([0.19032649, 0.24630058, 0.11542459, 0.26710801, 0.46347487,
       0.29537546, 0.42509999, 0.31049949, 0.34741666, 0.33224694,
       0.47575159, 0.23498767, 1.43352778, 1.01737958])

In [32]:
# Compute the average relative Brier score
average_normalized_brier = np.mean(combined_normalized_brier)
print("Normalized unifying score:", average_normalized_brier)

Normalized unifying score: 0.4396371224819819
